In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
tpm_file = pd.read_csv('base_data/Allsamples_tpm.csv')
tpm_file.head()

FB_Gene  S1_counts   S2_counts   S3_counts  S4_counts  S5_counts  \
0  FBgn0031081   3.557237    0.010032    0.027937   0.000000   0.021195   
1  FBgn0052826   0.000000    0.000000    0.000000   0.000000   0.000000   
2  FBgn0031085   5.224219    0.000000    0.110539   0.000000   0.000000   
3  FBgn0062565  59.890682  113.678741  106.025847  50.292973  84.532776   
4  FBgn0031088   1.449757    0.015499    0.014386   0.000000   0.000000   

   S6_counts  S7_counts  S8_counts  S9_counts  
0   0.000000   0.000000   0.136597   0.000000  
1   0.000000   0.000000   0.000000   0.000000  
2   0.000000   0.000000   0.000000   0.000000  
3  66.951918  36.736645  72.550286  52.186025  
4   0.000000   0.000000   0.000000   0.000000

~~S1: Control, Replicate 1~~ - Outlier

S2: 20E, Replicate 1

S3: IMD (HKSM), Replicate 1

S4: Control, Replicate 2

S5: 20E, Replicate 2

S6: IMD (HKSM), Replicate 2

S7: Control, Replicate 3

S8: 20E, Replicate 3

S9: IMD (HKSM), Replicate 3

In [3]:
logfc_df = pd.DataFrame()
logfc_df['Gene'] = tpm_file['FB_Gene']

# Control Average
logfc_df['ctrl_average'] = (tpm_file['S4_counts'] + tpm_file['S7_counts'])/2

# 20E average
logfc_df['20e_average'] = (tpm_file['S2_counts'] + tpm_file['S5_counts'] + tpm_file['S8_counts'])/3

# IMD average
logfc_df['imd_average'] = (tpm_file['S3_counts'] + tpm_file['S6_counts'] + tpm_file['S9_counts'])/3
logfc_df

Gene  ctrl_average  20e_average  imd_average
0      FBgn0031081      0.000000     0.055941     0.009312
1      FBgn0052826      0.000000     0.000000     0.000000
2      FBgn0031085      0.000000     0.000000     0.036846
3      FBgn0062565     43.514809    90.253934    75.054597
4      FBgn0031088      0.000000     0.005166     0.004795
...            ...           ...          ...          ...
17601  FBgn0031288      0.625079     0.209066     0.190154
17602  FBgn0031289      0.573550     0.217697     0.069054
17603  FBgn0002936      0.000000     0.000000     0.000000
17604  FBgn0002563      0.086863     0.000000     0.005478
17605  FBgn0046113      0.000000     0.000000     0.005738

[17606 rows x 4 columns]

In [4]:
# CTRL vs 20E fold change
logfc_df['logFC_20EvsCTRL'] = np.log1p(logfc_df['20e_average'] / logfc_df['ctrl_average'])

# CTRL vs IMD fold change
logfc_df['logFC_IMDvsCTRL'] = np.log1p(logfc_df['imd_average'] / logfc_df['ctrl_average'])

# 20E vs IMD fold change
logfc_df['logFC_20EvsIMD'] = np.log1p(logfc_df['20e_average'] / logfc_df['imd_average'])

# Replace infinities and NaNs with 0 across all logFC columns
logfc_df[['logFC_20EvsCTRL','logFC_IMDvsCTRL','logFC_20EvsIMD']] = (
    logfc_df[['logFC_20EvsCTRL','logFC_IMDvsCTRL','logFC_20EvsIMD']]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
)